In [1]:
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *

In [2]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [3]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4a4f0c4a-0d0c-4201-ac6c-19756581a39d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 71ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [4]:
loader = IncrementalLoader(spark, '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma', 'delta_news/sports')
df = loader.get_new_data()

25/05/02 18:04:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


CDF not available — doing full load


25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 1 with a timestamp 1745921639653 which is greater than the next commit timestamp 1745921639219.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639654 which is greater than the next commit timestamp 1745921639603.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639655 which is greater than the next commit timestamp 1745921639603.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 4 with a timestamp 1745921639656 which is greater than the next commit timestamp 1745921639653.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639657 which is greater than the next commit timestamp 1745921639445.
25/05/02 18:04:41 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639658 which is greater than the next commit timestamp 1745921639603.
25/05/02 18:04:41 WARN DeltaHistoryManager: Fo

In [10]:
processor = NewsProcessor(spark, df)

In [11]:
processor.remove_clear_duplicates()
processor.name_to_id()
processor.remove_hidden_duplicates(['url'])
processor.ensure_schema()

Removed 46 simple duplicate(s)
Removed 1 hidden duplicate(s)


In [13]:
processor.df

url,author,content,description,publishedAt,source,title,urlToImage
http://deadline.c...,Dominic Patten,Both lawyers for ...,The ESPN host say...,2025-04-23 03:32:00,"{deadline, Deadline}",Shannon Sharpe Ad...,https://deadline....
http://deadline.c...,Natalie Oganesyan,With a hometown v...,With a hometown v...,2025-04-27 05:05:00,"{deadline, Deadline}",Wrexham Promoted ...,https://deadline....
http://pitcherlis...,Nick Pollack,Welcome to the SP...,Nick Pollack revi...,2025-04-16 09:35:04,"{pitcherlist.com,...",Fantasy Baseball ...,https://pitcherli...
http://www.hollyw...,Alex Weprin,ESPNFirst Take ho...,Smith said on his...,2025-04-26 01:58:33,{hollywood-report...,Stephen A. Smith ...,https://www.holly...
https://247sports...,Chris Hummer and ...,To read this full...,After the 2025 co...,2025-04-17 16:37:30,"{247sports, 247Sp...",Transfer portal i...,https://s3media.2...
https://247sports...,Chris Hummer and ...,The 2025 college ...,From updates on t...,2025-04-15 17:05:24,"{247sports, 247Sp...",Transfer portal's...,https://s3media.2...
https://247sports...,Kipp Adams,The 2025 NFL Draf...,The Dawgs once ag...,2025-04-28 01:28:13,"{247sports, 247Sp...",Two Georgia footb...,https://s3media.2...
https://247sports...,Adam Smith,UNC is losing its...,Were profiling th...,2025-04-16 05:00:37,"{247sports, 247Sp...",UNC Football Tran...,https://s3media.2...
https://247sports...,Patrick Brown,The offseason for...,GoVols247 puts a ...,2025-04-22 23:11:15,"{247sports, 247Sp...",X’s and O’s: Brea...,https://s3media.2...
https://247sports...,Brad Crawford,The 2025 NFL Draf...,Several allconfer...,2025-04-27 16:03:45,"{247sports, 247Sp...",2025 NFL Draft's ...,https://s3media.2...
